# **5. Build the Neural Network**

Neural network는 데이터에 대해 여러 연산을 수행하는 layer로 구성됨

torch.nn 은 신경망을 직접 구축하는데 필요한 모든 기본 구성 요소들을 제공

신경망 또한 하나의 모듈이고 그 안에 여러 개의 모듈(레이어)를 포함하는 구조

이러한 중첩 구조(nested structure) 덕분에 복잡한 신경망 아키텍쳐도 효율적으로 구성하고 관리할 수 있음

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


# Define the Class


In [ ]:
# Pytorch 에서 모든 신경망은 nn.Module을 상속해서 만듬
class NeuralNetwork(nn.Module):
  # 네트워크 구조를 정의
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
    )

  # 입력 데이터의 흐름을 정의
  # x를 flatten하고 쌓아둔 layer를 통과시킴
  # 그리고 logits(소프트맥스 전에 출력되는 벡터)을 반환
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
X = torch.rand(1, 28, 28, device=device) # 입력 이미지 하나 생성 (랜덤값으로 만드니까)
logits = model(X) # 모델에 입력을 넣어서 logits 계산
pred_probab = nn.Softmax(dim=1)(logits) # softmax로 각 클래스별 확률 계산
y_pred = pred_probab.argmax(1) # 가장 높은 확률을 가진 클래스를 예측 결과로
print(f"predicated class: {y_pred}")

predicated class: tensor([1])


# Model Layers

In [ ]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


# nn.Flatten

flatten을 통해 이미지 3장을 3,784로 바꿈

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


# nn.Linear

입력 784 -> 20 으로 선형 변환 시킴

In [ ]:
layer1 = nn.Linear(in_features = 28*28, out_features = 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


# nn.ReLU

ReLU 이후 값을 확인해 보면 음수값은 0으로 되고 양수값은 그대로 유지되는 것을 확인할 수 있음

In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.5496, -0.5485, -0.3551,  0.0160, -0.5700,  0.2855, -0.6189,  0.7105,
          0.0067,  0.1517, -0.0197,  0.1228,  0.0251,  0.8445, -0.0536, -0.0212,
          0.2953,  0.7620,  0.0828, -0.2206],
        [-0.4959, -0.2937, -0.1727, -0.1377, -0.5198,  0.2325, -0.0708,  0.5165,
          0.0914,  0.1246,  0.0165,  0.1722,  0.0802,  0.3802,  0.2920,  0.1719,
          0.3584,  0.3179,  0.4968, -0.5367],
        [-0.2523, -0.0689, -0.1692,  0.0777, -0.2076,  0.3334, -0.0826,  1.0497,
          0.1373,  0.4599, -0.2655,  0.2997,  0.0515,  0.3888, -0.0869, -0.1003,
          0.2010,  0.2358,  0.1786, -0.1773]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0160, 0.0000, 0.2855, 0.0000, 0.7105, 0.0067,
         0.1517, 0.0000, 0.1228, 0.0251, 0.8445, 0.0000, 0.0000, 0.2953, 0.7620,
         0.0828, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2325, 0.0000, 0.5165, 0.0914,
         0.1246, 0.0165, 0.1722, 0.0802, 0.3802, 0.29

# nn.Sequential


레이어들의 컨테이너 역할을 하는것 시퀀셜이

logits을 통해 예측까지 가능하게 끔 준비

In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

# nn.Softmax

logts을 각 샘플마다 클래스의 확률 합이 1이 되게끔 softmax

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# Model Parameters

In [ ]:
# 단순하게 모델의 전체 구조를 출력
print(f"model structure: {model}\n\n")

# 각 파라미터에 대해 이름과 값의 쌍을 반환
for name, param in model.named_parameters():
  print(f"layer: {name} | size: {param.size()} | values: {param[:2]}\n")

model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


layer: linear_relu_stack.0.weight | size: torch.Size([512, 784]) | values: tensor([[ 0.0087,  0.0077,  0.0164,  ..., -0.0094,  0.0229, -0.0122],
        [-0.0309,  0.0323, -0.0281,  ..., -0.0068,  0.0198, -0.0291]],
       grad_fn=<SliceBackward0>)

layer: linear_relu_stack.0.bias | size: torch.Size([512]) | values: tensor([ 0.0320, -0.0032], grad_fn=<SliceBackward0>)

layer: linear_relu_stack.2.weight | size: torch.Size([512, 512]) | values: tensor([[ 0.0194,  0.0292, -0.0172,  ..., -0.0213, -0.0214, -0.0324],
        [-0.0250,  0.0344, -0.0167,  ...,  0.0345, -0.0296,  0.0117]],
       grad_fn=<SliceBackward0>)

layer: linear_relu_stack.2.bias | size: 